## Data Cleanup & Analysis

* We performed ETL (Extract Transform Load)  from a Netflix database obtained from Kaggle (https://www.kaggle.com/shivamb/netflix-shows/version/3). Since the database already give us the title and the cast, we made a similar database but with the actor/actress and all the movie titles they appear. We used Python to clean and prepare the data. We made three datasets to save the cast with an id number, titles with their show id (already given) and a new set with the separated cast and the movie title they appear. These datasets are exported to SQL and we made a new database to visualize each actor/actress and their respective movie titles. 

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
# Import csv data and visualize how is the information
file = "netflix_titles.csv"
data = pd.read_csv(file)
data.head()

#### We observed that the data has some missing (NaN) values. To load this information into SQL we removed these values. Also, the column names have an acceptable format, so we kept the information.

In [ ]:
# Delete NaN values
clean_data = data.replace(np.NaN)

#### We analyzed only the movie titles, so we extracted these information.

In [ ]:
# Only get the Movies data and visualize the information
movies_cast = clean_data[clean_data["type"] == 'Movie']
movies_cast

#### Now that the data is ready we got all the actors and actress appering in each title. We used the split function to separate all cast and saved the information on a list.

In [ ]:
# Now we get the actors that are on our data set
cast_list_movie = [] # Actors

for listed_in in movies_cast['cast']:
    split_cast = [x.strip() for x in listed_in.split(',')] # Split in list if there are more than one actor 
    # Save data
    for j in range(len(split_cast)): 
        cast_list_movie.append(split_cast[j].strip()) 

In [ ]:
cast_list_movie

#### Since the cast does not have an id number, we assigned it and make a dataframe with this information.

In [ ]:
# Define an array with numbers to asign as id to each actor/actress
id_cast = np.arange(len(cast_list_movie))

In [ ]:
# Dataframe from actors
no_cast = pd.DataFrame({'id_cast': id_cast, 'actor_actress': cast_list_movie}).set_index('id_cast')
no_cast

#### The movie titles already have an id, so we only made the new dataframe.

In [ ]:
# The movie titles already have an ID so we only make a DF
no_title = pd.DataFrame({'id_movie': movies_cast['show_id'], 'title': movies_cast['title']}).set_index('id_movie')
no_title

#### Now, we made a new dataframe based on the separated cast and their titles. We have a lot of repeated information in the dataframe but we get the results using this method.

In [ ]:
# Since we want to extract the actors and their movies, we are going to separate each each one and make a DB
# Cast and respective movies
ctemp = movies_cast['cast'].values
ttemp = movies_cast['title'].values

movie = []
cast = []

for i in range(len(ctemp)): # Row
    cstmp=ctemp[i].split(',') # Separate the data if there is more than one actor
    for k in range(len(cstmp)): # We analyze each actor in the selected row
        # Append the movie and respective actor/actress
        movie.append(ttemp[i])
        cast.append(cstmp[k].strip())

In [ ]:
# Make DB
df_movie_cast = pd.DataFrame({'title': movie, 'actor_actress': cast}).set_index('actor_actress')
df_movie_cast

## Connect to local Database

* To made our new database (with name of 'netflix_castmovie_db'), we used a connection to PostgreSQL, confirmed the tables are created and send the datasets we made before.

In [ ]:
# Create Database Connection
connection_string = "postgres:postgres@localhost:5432/netflix_castmovie_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
# Since we already create de Tables in our query
# Send data to SQL
# This is for cast id
no_cast.to_sql(name='no_cast', con=engine, if_exists='append', index=True)

In [ ]:
# For titles id
no_title.to_sql(name='no_title', con=engine, if_exists='append', index=True)

In [ ]:
# And the cast separated with their respective movie titles.
df_movie_cast.to_sql(name='title_cast', con=engine, if_exists='append', index=True)

## Confirm data has been added by querying the tables.

* In pgAdmin we confirmed that the data has been added to our new database called 'netflix cast id'. Here, we made a second check and visualize each actor and actress with all the movies they appear.

In [ ]:
netflix_casting = pd.read_sql_query('SELECT * FROM netflix_cast_id', con=engine)
netflix_casting.head(10)

In [ ]:
# If we want to know about a certain person...
netflix_casting.loc[netflix_casting['actor_actress'] == 'Anthony Hopkins']